In [1]:
import os, sys, tempfile, logging, time
import csv
from datetime import date
from dscollaborative.recommender import UserItemRatingMatrix, ImplicitModel, Features, Cleaner, Duplicates
from dscollaborative.recommender import reranker_x_and_y, reranker_x_only
from dstools.utils import normalize_path, save_list_to_file, file_to_list
import json
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import sparse
import scipy
import pickle
import random
import time


logging.basicConfig(level=logging.INFO)

In [2]:
with open("kids_character_mapping.pkl", "rb") as fp:   #Pickling
    character_sids_dict = pickle.load(fp)
character_sids_dict

{'BLEACH': ['SID0003144'],
 'BORUTO': ['SID0029042', 'SID0053510'],
 'DAYS': ['SID0025580', 'SID0054283'],
 'DRAGON QUEST ダイの大冒険': ['SID0027069',
  'SID0044873',
  'SID0050918',
  'SID0054518',
  'SID0054519',
  'SID0054520'],
 'EDENS ZERO': ['SID0056092'],
 'FAIRY TAIL': ['SID0003079',
  'SID0029354',
  'SID0032234',
  'SID0037381',
  'SID0057458'],
 'HUNTER×HUNTER': ['SID0002812', 'SID0002956', 'SID0011645'],
 'LEGO': ['SID0017681',
  'SID0017704',
  'SID0017986',
  'SID0022373',
  'SID0024363',
  'SID0025485',
  'SID0026085',
  'SID0026086',
  'SID0027621',
  'SID0027622',
  'SID0028869',
  'SID0029280',
  'SID0029843',
  'SID0029844',
  'SID0029967',
  'SID0032303',
  'SID0032352'],
 'NARUTO': ['SID0014715',
  'SID0016111',
  'SID0017723',
  'SID0017724',
  'SID0017725',
  'SID0017726',
  'SID0017727',
  'SID0017780',
  'SID0017816',
  'SID0017817',
  'SID0017818',
  'SID0017819',
  'SID0017820',
  'SID0017821',
  'SID0017822',
  'SID0017823',
  'SID0017824',
  'SID0017825',
  'SID

In [17]:
sid_set = set()
for v in character_sids_dict.values():
    sid_set.update(set(v))
len(sid_set)

1385

In [3]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [54]:
class Ranker:
    def __init__(self, model_path):
        self.collab_model = ImplicitModel()
        self.collab_model.load_model(model_path)

    def rank(self, target_users, target_items, N=200, filter_already_liked_items=True, batch_size=10000):
        """
        :param target_users: UID list, [uid1, uid2, ...]; None means all users in matrix
        :param target_items: SID list  [sid1, sid2, ...]; None means all items in matrix
        :param N: minimum nb of output
        :yield: user_id, reco list [SID1, SID2, ...]
        """
        # filtering out users who are not in matrix
        filtered_users = self.collab_model.filter_users(target_users)
        filtered_users_ids = list(map(lambda x: self.collab_model.model.user_item_matrix.user2id[x], filtered_users))

        # filtering out items who are not in matrix
        filtered_items = self.collab_model.filter_items(target_items)
        filtered_items_ids = list(map(lambda x: self.collab_model.model.user_item_matrix.item2id[x], filtered_items))
        item_vector = self.collab_model.model.item_factors.take(filtered_items_ids, axis=0)

        if len(filtered_users_ids) == 0 or len(filtered_items_ids) == 0:
            logging.debug("not filtered_users_ids or not item_vector")
            return None, None

        if filter_already_liked_items:
            # seen matrix
            filtered_seen_matrix = self.collab_model.model.user_item_matrix.matrix[:, filtered_items_ids]

            for sub_user_list in batch(filtered_users_ids, batch_size):
                sub_users = self.collab_model.model.user_factors.take(sub_user_list, axis=0)
                subset_users_seen_items = filtered_seen_matrix[sub_user_list, :].todense()

                ordered_indexes, first_item_seen_index = reranker_x(sub_users, item_vector, filtered_items_ids,
                                                                    subset_users_seen_items, filter_seen=True)

                for user_index, reco in zip(sub_user_list, self.collab_model.defactorize_sids(ordered_indexes)):
                    yield self.collab_model.model.user_item_matrix.id2user[user_index], list(reco)[:N]
        else:
            for sub_user_list in batch(filtered_users_ids, batch_size):
                sub_users = self.collab_model.model.user_factors.take(sub_user_list, axis=0)
                ordered_indexes, _ = reranker_x(sub_users, item_vector, filtered_items_ids)
                for user_index, reco in zip(sub_user_list, self.collab_model.defactorize_sids(ordered_indexes)):
                    yield self.collab_model.model.user_item_matrix.id2user[user_index], list(reco)[:N]
    
    def get_users_in_model(self, target_users):
        filtered_users = self.collab_model.filter_users(target_users)
        print(f"{len(target_users)} -> {len(filtered_users)}")
        return filtered_users
    
    def fet_score_(self, target_users, target_items, seen_SID_weight=False, batch_size=10000):
        """
        :param target_users: UID list, [uid1, uid2, ...]; None means all users in matrix
        :param target_items: SID list  [sid1, sid2, ...]; None means all items in matrix
        :param N: minimum nb of output
        :yield: user_id, reco list [SID1, SID2, ...]
        """
        # filtering out users who are not in matrix
        filtered_users = self.collab_model.filter_users(target_users)
        filtered_users_ids = list(map(lambda x: self.collab_model.model.user_item_matrix.user2id[x], filtered_users))

        # filtering out items who are not in matrix
        filtered_items = self.collab_model.filter_items(target_items)
        filtered_items_ids = list(map(lambda x: self.collab_model.model.user_item_matrix.item2id[x], filtered_items))
        item_vector = self.collab_model.model.item_factors.take(filtered_items_ids, axis=0)

        if len(filtered_users_ids) == 0 or len(filtered_items_ids) == 0:
            logging.debug("not filtered_users_ids or not item_vector")
            return None, None
        
        for sub_user_list in batch(filtered_users_ids, batch_size):
            user_vectors = self.collab_model.model.user_factors.take(sub_user_list, axis=0)
            scores = user_vectors.dot(item_vector.T)
            yield np.mean(scores, axis=1)
    
    def fet_score(self, target_users, target_items, seen_SID_weight=True, batch_size=10000):
        """
        :param target_users: UID list, [uid1, uid2, ...]; None means all users in matrix
        :param target_items: SID list  [sid1, sid2, ...]; None means all items in matrix
        :param N: minimum nb of output
        :yield: user_id, reco list [SID1, SID2, ...]
        """
        # filtering out users who are not in matrix
        filtered_users = self.collab_model.filter_users(target_users)
        filtered_users_ids = list(map(lambda x: self.collab_model.model.user_item_matrix.user2id[x], filtered_users))

        # filtering out items who are not in matrix
        filtered_items = self.collab_model.filter_items(target_items)
        filtered_items_ids = list(map(lambda x: self.collab_model.model.user_item_matrix.item2id[x], filtered_items))
        item_vector = self.collab_model.model.item_factors.take(filtered_items_ids, axis=0)

        if len(filtered_users_ids) == 0 or len(filtered_items_ids) == 0:
            logging.debug("not filtered_users_ids or not item_vector")
            return None, None
        
        if seen_SID_weight:
            filtered_seen_matrix =  self.collab_model.model.user_item_matrix.matrix[:, filtered_items_ids]

        
        for sub_user_list in batch(filtered_users_ids, batch_size):
            user_vectors = self.collab_model.model.user_factors.take(sub_user_list, axis=0)
            scores = user_vectors.dot(item_vector.T)
            if seen_SID_weight:
                subset_users_seen_items = filtered_seen_matrix[sub_user_list, :].todense()
                rows, cols = np.nonzero(subset_users_seen_items)
                scores[rows,cols] = scores[rows,cols] + 100
                
            yield np.mean(scores, axis=1)


In [82]:
def rank_characters(target_user, output_sid=False, seen_SID_weight=False):
    model_users = model.get_users_in_model(target_user)
    character_scores = []
    r_character_list = []
    sid_list = []
    # have to make sure all users are in the model before
    for character, sids in character_sids_dict.items(): 
        scores = np.array([])
        for batch_score in model.fet_score(model_users, sids, seen_SID_weight=seen_SID_weight):
            scores = np.concatenate((scores, batch_score), axis=None)

        # skip empty SIDs
        if scores.size != 0:
            character_scores.append(scores)
            r_character_list.append(character)
            sid_list.append(sids[0])
    
    character_scores = np.array(character_scores).transpose()
    # sort from sm index to smallest index
    ranked_indice =  np.argsort(character_scores, axis=1)
    r_character_list = np.array(r_character_list)
    sid_list = np.array(sid_list)
    
    if output_sid:  # for demo 
        return ','.join(sid_list[ranked_indice[0][::-1]])
    else:
        # return r_character_list[ranked_indice]
        return r_character_list[ranked_indice[0][::-1]] # for demo

In [55]:
model = Ranker('als_model_w3.2021-12-07')
rank_characters(['fan_アンパンマン'])

1 -> 1


array(['アンパンマン', 'しまじろう', 'パウ・パトロール', 'ケロロ軍曹', 'おさるのジョージ', 'おしりたんてい',
       'プリキュア', 'ペッパピッグ＆パジャマスク-', 'HUNTER×HUNTER', 'いないいないばあっ！',
       'トムとジェリー', 'ワールドトリガー', 'ゲゲゲの鬼太郎', 'ペネロペ', 'おそ松くん', '暗殺教室', 'はなかっぱ',
       '弱虫ペダル', 'コラショ', 'ミッキーマウス', 'かいけつゾロリ', 'アナと雪の女王', '僕のヒーローアカデミア',
       'おジャ魔女どれみ', '美少女戦士セーラームーン', 'ミニオンズ', '忍たま乱太郎', 'きかんしゃトーマス',
       'トイストーリー', 'チャギントン', 'けろけろけろっぴ', 'SONIC', 'ハローキティ', 'ドラゴンボール',
       'のりものまん モービルランドのカークン', 'ミッフィー', 'ルルロロ', 'ポケットモンスター', '食戟のソーマ',
       'ルパン3世', 'DAYS', 'ティニー', 'ウサハナ', 'こちら葛飾区亀有公園前派出所', 'デジモンアドベンチャー',
       '犬夜叉', 'ガンダム', 'リトルツインスターズ(キキ＆ララ)', 'カンフー・パンダ', 'キン肉マン',
       'リロ＆スティッチ', 'モンスターズインク', 'ウルトラマン', 'キューティーハニー', 'くまのプーさん', 'キテレツ',
       'タイムボカン', 'みいつけた！', '銀河鉄道999', '鉄腕アトム', 'かいじゅうステップ\u3000ワンダバダ',
       'コナン', 'イナズマイレブン', '天才バカボン', 'おかあさんといっしょ', 'テニスの王子様', 'タマ＆フレンズ',
       'ワンピース', '妖怪ウォッチ', 'コボちゃん', 'PUI PUI モルカー', 'ベイブレード', 'WASIMO',
       '鷹の爪', 'メジャー', '聖闘士星矢', 'パズドラ', 'カピバラさん', 'シナモン', 'NARUTO',
       'ハイキュー!!', '

In [64]:
target_users = []
for line in efficient_reading("../data/superusers.csv"):
    target_users.append(line.rstrip())

target_users[:5]

['CM020027571', 'PM011378827', 'PM011697362', 'P0001491232', 'P0000687634']

In [67]:
ranked_characters = rank_characters(target_users)
ranked_characters.shape

47292 -> 4172


(4172, 129)

In [76]:
ranked_characters[8]

array(['食戟のソーマ', 'FAIRY TAIL', 'レイトン教授', '天才バカボン', 'キテレツ', 'ウサビッチ',
       'イナズマイレブン', '七つの大罪', 'ルパン3世', '聖闘士星矢', '銀河鉄道999', 'キン肉マン',
       'キューティーハニー', 'ガンダム', '鷹の爪', 'コボちゃん', '幽☆遊☆白書', 'ドゲンジャーズ', '犬夜叉',
       'タイムボカン', 'カードキャプターさくら', 'デュエル･マスターズ', 'バッドばつ丸', 'るろうに剣心－明治剣客浪漫譚－',
       'ベイブレード', 'PUI PUI モルカー', 'カピバラさん', '仮面ライダー', 'タマ＆フレンズ', '少年アシベ ',
       'DRAGON QUEST ダイの大冒険', 'スーパー戦隊', 'プリティーシリーズ', 'パズドラ', 'ワンピース',
       '鉄腕アトム', 'コナン', 'メジャー', 'アイカツ', 'こちら葛飾区亀有公園前派出所', 'ぼのぼの', 'タンタン',
       'リトルツインスターズ(キキ＆ララ)', '遊☆戯☆王', 'おじゃる丸', 'EDENS ZERO', 'マイメロディ',
       'ブラッククローバー', 'デジモンアドベンチャー', 'ちびまる子ちゃん', 'NARUTO', 'おそ松くん', 'ケロロ軍曹',
       'SLAM DUNK', 'WASIMO', 'SONIC', 'あたしンち', 'BLEACH', '美少女戦士セーラームーン',
       'マジカパーティー', 'あひるの空', 'テニスの王子様', 'ゲゲゲの鬼太郎', 'ミュークルドリーミー', '忍たま乱太郎',
       'ウサハナ', 'DAYS', 'けろけろけろっぴ', '黒子のバスケ', '暗殺教室', '炎炎ノ消防隊', 'LEGO',
       'ハローキティ', 'シナモン', 'かいけつゾロリ', 'ドラゴンボール', 'ウルトラマン',
       'かいじゅうステップ\u3000ワンダバダ', '弱虫ペダル', 'ムーミン', 'おジャ魔女どれみ', 'こびと',
       'Ｄｒ．ＳＴＯ

In [35]:
model = Ranker('als_model_w3.2021-12-07')

In [20]:
target_users = ['fan_NARUTO']
target_items = list(sid_set) # character_sids_dict['NARUTO']
# target_items[:5]

In [21]:
filtered_items = model.collab_model.filter_items(target_items)
filtered_items_ids = list(map(lambda x: model.collab_model.model.user_item_matrix.item2id[x], filtered_items))
item_vector = model.collab_model.model.item_factors.take(filtered_items_ids, axis=0)
item_vector.shape

(1183, 128)

In [22]:
filtered_users = model.collab_model.filter_users(target_users)
filtered_users_ids = list(map(lambda x: model.collab_model.model.user_item_matrix.user2id[x], filtered_users))
user_vectors = model.collab_model.model.user_factors.take(filtered_users_ids, axis=0)
user_vectors.shape

(1, 128)

In [23]:
scores = user_vectors.dot(item_vector.T)
scores.shape

(1, 1183)

In [24]:
scores

array([[ 2.5868137e-13, -4.6906429e-13,  1.8524882e-13, ...,
         8.9576255e-14,  5.9085896e-13,  2.9313739e-13]], dtype=float32)

In [25]:
np.mean(scores, axis=1)

array([7.240871e-14], dtype=float32)

In [26]:
list(model.fet_score(target_users, character_sids_dict['NARUTO']))

[array([-2.4576285e-14], dtype=float32)]

### add weight to seen SIDs

In [30]:
filtered_seen_matrix =  model.collab_model.model.user_item_matrix.matrix[filtered_users_ids, filtered_items_ids]
filtered_seen_matrix

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [31]:
rows, cols = np.nonzero(filtered_seen_matrix)
print(len(cols))
print(len(rows))

7
7


In [32]:
scores[rows,cols] = scores[rows,cols] + 100

In [33]:
scores

array([[ 2.5868137e-13, -4.6906429e-13,  1.8524882e-13, ...,
         8.9576255e-14,  5.9085896e-13,  2.9313739e-13]], dtype=float32)

## batch processing on fake data

In [62]:
def efficient_reading(input_path, with_header=True, header_format=None):
    """
    yield one line at once til go through entire file,

    & check the header_format
    """
    with open(input_path, 'r') as r:
        if with_header:
            header = r.readline().rstrip()
            logging.debug(f"reading file whose format is  {header}")
            if header_format:
                assert header == header_format, f"Header Format is WRONG"
        while True:
            line = r.readline()
            if line:
                yield line
            else:
                break

['CM020027571', 'PM011378827', 'PM011697362', 'P0001491232', 'P0000687634']

In [73]:
target_users = ['fan_アンパンマン', 'fan_妖怪ウォッチ', 'fan_NARUTO', 'fan_ハローキティ']

In [74]:
model_users = model.get_users_in_model(target_users)

4 -> 4


In [77]:
character_scores = []
r_character_list = []
# have to make sure all users are in the model before
for character, sids in character_sids_dict.items(): 
    scores = np.array([])
    for batch_score in model.fet_score(model_users, sids):
        scores = np.concatenate((scores, batch_score), axis=None)
    
    # skip empty SIDs
    if scores.size != 0:
        character_scores.append(scores)
        r_character_list.append(character)

character_scores = np.array(character_scores).transpose()
character_scores.shape

(4, 119)

In [78]:
# sort from sm index to smallest index
ranked_indice =  np.argsort(character_scores, axis=1)
ranked_indice

array([[ 66,  10,  67,  92,  43,  18,   8,  30, 101,  55,  80,  68,  84,
         62, 115,  86,  47,  95,  16,  70,  89,  15,  97,   0,  31,  93,
         22,   3,   4,   1,  46,  44,   5,  39,  63,  45,  26,  72,  78,
         96,  87,  38, 112,  77,  76,  59,  41,  60, 111,  81,  83,  69,
         36,  91, 105,  33, 110,  29, 104,  58, 114,  27,  99, 113, 103,
         73,  53,  24, 102,  23,   7,  56,  42,  85, 116,  88, 100,   9,
         20,  61,   6,  49,  34,  21,  40,  28,  65, 106,  48,  64,  79,
        118,  14,  54,  35, 109, 117,  90,  82,  52,  12,  13,  98,  51,
         17,   2, 107,  75,  50,  11, 108,  74,  32,  25,  19,  71,  94,
         37,  57],
       [109,  15,  17,  90,  54,  40,  67,  10,  44,  33,  11, 101,  76,
        116,  94,   3,  79,  48,  56,  29,  51,  25, 115,   9,  39,  82,
         98,  88,  24,   2,  66,  20,  22,  47,  72,   6, 113,  83, 114,
        112,  85,  19,  89,   4,  62,  18,  74,  91,  93, 105,  52, 117,
         59,   7,  32,  26,  75,

In [79]:
# sort from sm index to smallest index
ranked_indice =  np.argsort(character_scores, axis=1)
ranked_indice

array([[ 66,  10,  67,  92,  43,  18,   8,  30, 101,  55,  80,  68,  84,
         62, 115,  86,  47,  95,  16,  70,  89,  15,  97,   0,  31,  93,
         22,   3,   4,   1,  46,  44,   5,  39,  63,  45,  26,  72,  78,
         96,  87,  38, 112,  77,  76,  59,  41,  60, 111,  81,  83,  69,
         36,  91, 105,  33, 110,  29, 104,  58, 114,  27,  99, 113, 103,
         73,  53,  24, 102,  23,   7,  56,  42,  85, 116,  88, 100,   9,
         20,  61,   6,  49,  34,  21,  40,  28,  65, 106,  48,  64,  79,
        118,  14,  54,  35, 109, 117,  90,  82,  52,  12,  13,  98,  51,
         17,   2, 107,  75,  50,  11, 108,  74,  32,  25,  19,  71,  94,
         37,  57],
       [109,  15,  17,  90,  54,  40,  67,  10,  44,  33,  11, 101,  76,
        116,  94,   3,  79,  48,  56,  29,  51,  25, 115,   9,  39,  82,
         98,  88,  24,   2,  66,  20,  22,  47,  72,   6, 113,  83, 114,
        112,  85,  19,  89,   4,  62,  18,  74,  91,  93, 105,  52, 117,
         59,   7,  32,  26,  75,

In [80]:
r_character_list = np.array(r_character_list)

In [91]:
r_character_list[ranked_indice[0][::-1]]

array(['スポンジ・ボブ', 'アンパンマン', 'ルルロロ', 'ハイキュー!!', 'おとうさんといっしょ', 'けろけろけろっぴ',
       'ぼのぼの', 'ピングー', '暗殺教室', 'あたしンち', 'ゲゲゲの鬼太郎', 'フィン＆ジェイク', '斉木楠雄のΨ難',
       'DAYS', 'おしりたんてい', 'コナン', '七つの大罪', 'いないいないばあっ！', 'あひるの空', 'コボちゃん',
       'マイメロディ', 'モンスターズインク', '鷹の爪', '炎炎ノ消防隊', 'アイカツ', 'シナモン',
       'おかあさんといっしょ', '黒子のバスケ', 'ベイブレード', 'ティニー', 'キン肉マン', '忍たま乱太郎',
       'テニスの王子様', 'しまじろう', 'ウサビッチ', 'かいけつゾロリ', 'るろうに剣心－明治剣客浪漫譚－', 'ケロロ軍曹',
       'LEGO', 'タマ＆フレンズ', 'おジャ魔女どれみ', 'WASIMO', '僕のヒーローアカデミア', 'ムーミン',
       '食戟のソーマ', 'ミッフィー', 'カピバラさん', 'スパイダーマン', 'NARUTO', 'きかんしゃトーマス',
       '妖怪ウォッチ', 'くまのプーさん', 'コラショ', 'パズドラ', '少年アシベ ', '遊☆戯☆王', '仮面ライダー',
       'こびと', '鉄腕アトム', 'スーパー戦隊', '幽☆遊☆白書', 'ちびまる子ちゃん', '犬夜叉', 'みいつけた！',
       '弱虫ペダル', 'リトルツインスターズ(キキ＆ララ)', 'アナと雪の女王', 'トムとジェリー', 'マジカパーティー',
       'ポケットモンスター', '美少女戦士セーラームーン', 'タイムボカン', 'ウルトラマン', 'セサミ',
       'ブラッククローバー', 'プリキュア', '聖闘士星矢', 'イナズマイレブン', 'ミュークルドリーミー', 'ワンピース',
       'プリティーシリーズ', 'ハローキティ', 'こちら葛飾区亀有公園前派出所', 'ガンダム', 'ダイヤのA', 'ウサハナ',
       

In [41]:
r_character_list[ranked_indice[0][::-1]]

array(['スポンジ・ボブ', 'アンパンマン', 'ルルロロ', 'ハイキュー!!', 'おとうさんといっしょ', 'けろけろけろっぴ',
       'ぼのぼの', 'ピングー', '暗殺教室', 'あたしンち', 'ゲゲゲの鬼太郎', 'フィン＆ジェイク', '斉木楠雄のΨ難',
       'DAYS', 'おしりたんてい', 'コナン', '七つの大罪', 'いないいないばあっ！', 'あひるの空', 'コボちゃん',
       'マイメロディ', 'モンスターズインク', '鷹の爪', '炎炎ノ消防隊', 'アイカツ', 'シナモン',
       'おかあさんといっしょ', '黒子のバスケ', 'ベイブレード', 'ティニー', 'キン肉マン', '忍たま乱太郎',
       'テニスの王子様', 'しまじろう', 'ウサビッチ', 'かいけつゾロリ', 'るろうに剣心－明治剣客浪漫譚－', 'ケロロ軍曹',
       'LEGO', 'タマ＆フレンズ', 'おジャ魔女どれみ', 'WASIMO', '僕のヒーローアカデミア', 'ムーミン',
       '食戟のソーマ', 'ミッフィー', 'カピバラさん', 'スパイダーマン', 'NARUTO', 'きかんしゃトーマス',
       '妖怪ウォッチ', 'くまのプーさん', 'コラショ', 'パズドラ', '少年アシベ ', '遊☆戯☆王', '仮面ライダー',
       'こびと', '鉄腕アトム', 'スーパー戦隊', '幽☆遊☆白書', 'ちびまる子ちゃん', '犬夜叉', 'みいつけた！',
       '弱虫ペダル', 'リトルツインスターズ(キキ＆ララ)', 'アナと雪の女王', 'トムとジェリー', 'マジカパーティー',
       'ポケットモンスター', '美少女戦士セーラームーン', 'タイムボカン', 'ウルトラマン', 'セサミ',
       'ブラッククローバー', 'プリキュア', '聖闘士星矢', 'イナズマイレブン', 'ミュークルドリーミー', 'ワンピース',
       'プリティーシリーズ', 'ハローキティ', 'こちら葛飾区亀有公園前派出所', 'ガンダム', 'ダイヤのA', 'ウサハナ',
       

# run on fake 200

In [96]:
model = Ranker('als_model_200')  # ('../data/als_model.2021-08-17')

In [97]:
rank_characters(['fan_アンパンマン'])

1 -> 1


array(['スポンジ・ボブ', 'アンパンマン', 'ルルロロ', 'ハイキュー!!', 'おとうさんといっしょ', 'けろけろけろっぴ',
       'ぼのぼの', 'ピングー', '暗殺教室', 'あたしンち', 'ゲゲゲの鬼太郎', 'フィン＆ジェイク', '斉木楠雄のΨ難',
       'DAYS', 'おしりたんてい', 'コナン', '七つの大罪', 'いないいないばあっ！', 'あひるの空', 'コボちゃん',
       'マイメロディ', 'モンスターズインク', '鷹の爪', '炎炎ノ消防隊', 'アイカツ', 'シナモン',
       'おかあさんといっしょ', '黒子のバスケ', 'ベイブレード', 'ティニー', 'キン肉マン', '忍たま乱太郎',
       'テニスの王子様', 'しまじろう', 'ウサビッチ', 'かいけつゾロリ', 'るろうに剣心－明治剣客浪漫譚－', 'ケロロ軍曹',
       'LEGO', 'タマ＆フレンズ', 'おジャ魔女どれみ', 'WASIMO', '僕のヒーローアカデミア', 'ムーミン',
       '食戟のソーマ', 'ミッフィー', 'カピバラさん', 'スパイダーマン', 'NARUTO', 'きかんしゃトーマス',
       '妖怪ウォッチ', 'くまのプーさん', 'コラショ', 'パズドラ', '少年アシベ ', '遊☆戯☆王', '仮面ライダー',
       'こびと', '鉄腕アトム', 'スーパー戦隊', '幽☆遊☆白書', 'ちびまる子ちゃん', '犬夜叉', 'みいつけた！',
       '弱虫ペダル', 'リトルツインスターズ(キキ＆ララ)', 'アナと雪の女王', 'トムとジェリー', 'マジカパーティー',
       'ポケットモンスター', '美少女戦士セーラームーン', 'タイムボカン', 'ウルトラマン', 'セサミ',
       'ブラッククローバー', 'プリキュア', '聖闘士星矢', 'イナズマイレブン', 'ミュークルドリーミー', 'ワンピース',
       'プリティーシリーズ', 'ハローキティ', 'こちら葛飾区亀有公園前派出所', 'ガンダム', 'ダイヤのA', 'ウサハナ',
       

In [98]:
rank_characters(['fan_妖怪ウォッチ'])  # 'fan_アンパンマン', 'fan_妖怪ウォッチ', 'fan_NARUTO', 'fan_ハローキティ'

1 -> 1


array(['妖怪ウォッチ', 'るろうに剣心－明治剣客浪漫譚－', 'BLEACH', 'タイムボカン', 'あひるの空',
       'シャーマンキング', 'トムとジェリー', 'ハイキュー!!', 'キテレツ', 'ミニオンズ', 'SLAM DUNK',
       '斉木楠雄のΨ難', 'おかあさんといっしょ', 'ワールドトリガー', 'プリティーシリーズ', 'ティニー',
       'スポンジ・ボブ', 'BORUTO', 'パズドラ', 'ダイヤのA', 'カピバラさん', 'ケロロ軍曹',
       'リロ＆スティッチ', 'ミッキーマウス', 'ドラゴンボール', '暗殺教室', '仮面ライダー',
       'のりものまん モービルランドのカークン', 'アナと雪の女王', '犬夜叉', 'HUNTER×HUNTER',
       'おしりかじり虫', 'ペネロペ', 'プリキュア', 'コラショ', 'タマ＆フレンズ', 'かいけつゾロリ', 'スーパー戦隊',
       'こびと', 'ポケットモンスター', 'ゲゲゲの鬼太郎', 'アイカツ', 'カンフー・パンダ', 'ミュークルドリーミー',
       'ガンダム', 'イナズマイレブン', 'テニスの王子様', '幽☆遊☆白書', 'トイストーリー', 'ウルトラマン',
       '少年アシベ ', 'いないいないばあっ！', 'きかんしゃトーマス', 'はなかっぱ', '僕のヒーローアカデミア',
       '忍たま乱太郎', 'しまじろう', '美少女戦士セーラームーン', '黒子のバスケ', 'ワンピース', 'アンパンマン',
       'レイトン教授', 'フィン＆ジェイク', 'こちら葛飾区亀有公園前派出所', 'ぼのぼの', 'NARUTO', 'セサミ',
       '鷹の爪', 'コボちゃん', '弱虫ペダル', 'ルパン3世', 'リトルツインスターズ(キキ＆ララ)', 'ピングー',
       'おそ松くん', 'タンタン', 'FAIRY TAIL', 'メジャー', 'おとうさんといっしょ', 'ミッフィー',
       '聖闘士星矢', '鉄腕アトム', 'マジカパーティー', '遊☆戯☆王',

In [99]:
rank_characters(['fan_NARUTO']) 

1 -> 1


array(['NARUTO', '食戟のソーマ', 'リトルツインスターズ(キキ＆ララ)', 'ティニー', 'マジカパーティー',
       'iiiあいすくりん', 'みいつけた！', 'ワールドトリガー', 'キテレツ', 'ミッフィー', 'BLEACH',
       '幽☆遊☆白書', 'ミュークルドリーミー', 'いないいないばあっ！', 'ゲゲゲの鬼太郎', 'テニスの王子様',
       'カンフー・パンダ', 'BORUTO', 'シナモン', 'ガンダム', 'おさるのジョージ', 'アナと雪の女王',
       'カードキャプターさくら', 'メジャー', '天才バカボン', 'デジモンアドベンチャー', 'こびと',
       '美少女戦士セーラームーン', '弱虫ペダル', 'DRAGON QUEST ダイの大冒険', '仮面ライダー',
       'おかあさんといっしょ', 'ポケットモンスター', '僕のヒーローアカデミア', 'けろけろけろっぴ', 'ハイキュー!!',
       'イナズマイレブン', 'WASIMO', 'カピバラさん', 'トムとジェリー', 'スポンジ・ボブ', 'ベイブレード',
       'モンスターズインク', 'ワンピース', '七つの大罪', 'ドラゴンボール', 'のりものまん モービルランドのカークン',
       'セサミ', '忍たま乱太郎', 'LEGO', 'キューティーハニー', 'ケロロ軍曹', 'スーパー戦隊', 'ウサビッチ',
       'アンパンマン', 'るろうに剣心－明治剣客浪漫譚－', 'ハローキティ', 'こちら葛飾区亀有公園前派出所', 'コナン',
       'かいけつゾロリ', 'FAIRY TAIL', 'ルパン3世', 'タイムボカン', 'プリキュア', 'アイカツ',
       'しまじろう', '少年アシベ ', 'ピングー', '暗殺教室', '妖怪ウォッチ', 'はなかっぱ', 'フィン＆ジェイク',
       'プリティーシリーズ', 'ウルトラマン', 'デュエル･マスターズ', 'タマ＆フレンズ', '聖闘士星矢',
       'きかんしゃトーマス', 'おしりたんてい', 'ぼのぼの'

In [100]:
rank_characters(['fan_ハローキティ']) 

1 -> 1


array(['ルルロロ', 'るろうに剣心－明治剣客浪漫譚－', 'DRAGON QUEST ダイの大冒険', 'あひるの空',
       'ハローキティ', 'ハイキュー!!', '鉄腕アトム', 'ミッキーマウス', '斉木楠雄のΨ難',
       'のりものまん モービルランドのカークン', 'マジカパーティー', '犬夜叉', 'カピバラさん', '少年アシベ ',
       'ムーミン', 'ガンダム', 'コボちゃん', 'タイムボカン', 'カンフー・パンダ', 'コラショ', 'アイカツ',
       'しまじろう', 'NARUTO', 'デュエル･マスターズ', 'ウサハナ', 'キューティーハニー', 'シャーマンキング',
       'けろけろけろっぴ', '天才バカボン', '遊☆戯☆王', 'ドラゴンボール', 'トムとジェリー', 'テニスの王子様',
       'おしりかじり虫', 'ワンピース', 'おしりたんてい', 'こびと', 'タマ＆フレンズ', 'ポケットモンスター',
       '幽☆遊☆白書', 'きかんしゃトーマス', 'キン肉マン', 'キテレツ', 'ウルトラマン', 'はなかっぱ', '聖闘士星矢',
       'メジャー', 'ミュークルドリーミー', 'ベイブレード', 'コナン', 'かいけつゾロリ', 'ピングー',
       'かいじゅうステップ\u3000ワンダバダ', 'トイストーリー', '忍たま乱太郎', 'パズドラ', 'フィン＆ジェイク',
       'ちびまる子ちゃん', 'おジャ魔女どれみ', 'ケロロ軍曹', 'リロ＆スティッチ', '黒子のバスケ', '銀河鉄道999',
       'ルパン3世', '仮面ライダー', 'HUNTER×HUNTER', 'こちら葛飾区亀有公園前派出所', 'あたしンち',
       'アナと雪の女王', 'プリティーシリーズ', 'BLEACH', 'いないいないばあっ！', 'アンパンマン',
       'iiiあいすくりん', 'FAIRY TAIL', '僕のヒーローアカデミア', 'ぼのぼの', 'WASIMO', 'LEGO',
       'ゲゲゲの鬼太郎', 'ウサビッチ', '

# run on fake + real data - bpr

In [102]:
model = Ranker('bpr_model.2021-12-06')  # 

In [112]:
rank_characters(['fan_アンパンマン'])  # 'fan_アンパンマン', 'fan_妖怪ウォッチ', 'fan_NARUTO', 'fan_ハローキティ', ドラゴンボール

1 -> 1


array(['パウ・パトロール', 'ペネロペ', 'しまじろう', 'チャギントン', 'ペッパピッグ＆パジャマスク-', 'アンパンマン',
       'きかんしゃトーマス', 'ルルロロ', 'おさるのジョージ', 'ミッフィー', 'おしりたんてい', 'はなかっぱ',
       'ピングー', 'のりものまん モービルランドのカークン', 'iiiあいすくりん', 'ミッキーマウス', 'シナモン',
       'いないいないばあっ！', 'けろけろけろっぴ', 'くまのプーさん', 'リトルツインスターズ(キキ＆ララ)', 'ティニー',
       'おしりかじり虫', 'ハローキティ', 'ウサハナ', 'コラショ', 'カピバラさん',
       'かいじゅうステップ\u3000ワンダバダ', 'リロ＆スティッチ', 'みいつけた！', 'おとうさんといっしょ',
       'WASIMO', 'おかあさんといっしょ', 'セサミ', 'こびと', 'ミュークルドリーミー',
       '宇宙なんちゃら こてつくん', 'ウサビッチ', 'アナと雪の女王', 'プリキュア', 'トムとジェリー', 'トイストーリー',
       '少年アシベ ', 'ミニオンズ', 'タマ＆フレンズ', 'かいけつゾロリ', 'ポケットモンスター', 'マイメロディ',
       'スポンジ・ボブ', 'ぼのぼの', 'ゲゲゲの鬼太郎', 'PUI PUI モルカー', 'ムーミン', 'SONIC',
       'モンスターズインク', 'おジャ魔女どれみ', 'おじゃる丸', 'マジカパーティー', 'ドゲンジャーズ', 'LEGO',
       'バッドばつ丸', 'ちびまる子ちゃん', '妖怪ウォッチ', 'ふしぎ駄菓子屋 銭天堂', '忍たま乱太郎',
       'プリティーシリーズ', 'ウルトラマン', 'キテレツ', 'コボちゃん', '美少女戦士セーラームーン', 'アイカツ',
       'あたしンち', '鉄腕アトム', 'ケロロ軍曹', 'パズドラ', 'スーパー戦隊', '天才バカボン', 'カンフー・パンダ',
       'タイムボカン', '仮面ライダー', 'キン肉マン', '

# run on fake + real data - als

x: NARUTO & ハローキティ

In [79]:
model = Ranker('als_model.2021-12-06')  # 

In [86]:
rank_characters(['fan_アンパンマン'], False, True)  # 'fan_アンパンマン', 'fan_妖怪ウォッチ', 'fan_NARUTO', 'fan_ハローキティ', fan_ドラゴンボール

1 -> 1


array(['アンパンマン', 'しまじろう', 'パウ・パトロール', 'おしりたんてい', 'おさるのジョージ', 'ケロロ軍曹',
       'ペッパピッグ＆パジャマスク-', 'プリキュア', '僕のヒーローアカデミア', 'いないいないばあっ！', 'トムとジェリー',
       'ゲゲゲの鬼太郎', 'おそ松くん', 'ペネロペ', '暗殺教室', 'かいけつゾロリ', 'ミニオンズ', 'コラショ',
       'ミッキーマウス', 'はなかっぱ', 'ワールドトリガー', 'チャギントン', 'きかんしゃトーマス', 'トイストーリー',
       'メジャー', 'おジャ魔女どれみ', 'イナズマイレブン', 'アナと雪の女王', '忍たま乱太郎', 'けろけろけろっぴ',
       'カンフー・パンダ', 'テニスの王子様', 'SONIC', 'ハローキティ', 'ミッフィー', 'ポケットモンスター',
       'DAYS', 'ドラゴンボール', 'のりものまん モービルランドのカークン', '美少女戦士セーラームーン',
       'デジモンアドベンチャー', 'ルルロロ', 'キン肉マン', '銀河鉄道999', 'モンスターズインク',
       'キューティーハニー', 'ティニー', 'ウサハナ', 'ベイブレード', 'HUNTER×HUNTER', 'タイムボカン',
       'リトルツインスターズ(キキ＆ララ)', 'リロ＆スティッチ', 'くまのプーさん', '七つの大罪', 'ダイヤのA',
       '妖怪ウォッチ', '食戟のソーマ', 'ルパン3世', 'みいつけた！', '天才バカボン', 'るろうに剣心－明治剣客浪漫譚－',
       'ワンピース', '弱虫ペダル', 'キテレツ', '鷹の爪', '遊☆戯☆王', 'ウルトラマン', 'ガンダム',
       '鉄腕アトム', '犬夜叉', 'デュエル･マスターズ', 'おかあさんといっしょ', 'タマ＆フレンズ',
       'PUI PUI モルカー', 'パズドラ', 'かいじゅうステップ\u3000ワンダバダ', 'コボちゃん', '聖闘士星矢',
       'こちら葛飾区亀有公園前派出所

In [57]:
rank_characters(['fan_妖怪ウォッチ'], False, True) 

1 -> 1


array(['妖怪ウォッチ', 'ポケットモンスター', 'ふしぎ駄菓子屋 銭天堂', 'おしりたんてい', 'マジカパーティー',
       'スポンジ・ボブ', 'かいけつゾロリ', 'イナズマイレブン', 'パウ・パトロール', 'トムとジェリー',
       '宇宙なんちゃら こてつくん', 'ペッパピッグ＆パジャマスク-', '暗殺教室', 'デュエル･マスターズ', 'コラショ',
       'こびと', 'SONIC', 'ゲゲゲの鬼太郎', 'ケロロ軍曹', 'ミニオンズ', '七つの大罪', 'LEGO',
       'おさるのジョージ', 'キテレツ', 'のりものまん モービルランドのカークン', 'ドラゴンボール', 'レイトン教授',
       'ベイブレード', 'おしりかじり虫', 'ウサビッチ', 'はなかっぱ', 'パズドラ', 'あたしンち', 'おそ松くん',
       'かいじゅうステップ\u3000ワンダバダ', '忍たま乱太郎', 'Ｄｒ．ＳＴＯＮＥ', 'プリキュア', '炎炎ノ消防隊',
       'ウルトラマン', 'カンフー・パンダ', '仮面ライダー', 'アイカツ', 'ミュークルドリーミー', '少年アシベ ',
       'スーパー戦隊', 'DRAGON QUEST ダイの大冒険', '鷹の爪', 'プリティーシリーズ', '僕のヒーローアカデミア',
       'iiiあいすくりん', '黒子のバスケ', 'WASIMO', 'しまじろう', 'デジモンアドベンチャー', 'ドゲンジャーズ',
       '天才バカボン', 'スパイダーマン', 'DAYS', 'おじゃる丸', 'チャギントン', 'メジャー', '弱虫ペダル',
       'キン肉マン', 'けろけろけろっぴ', '鉄腕アトム', 'NARUTO', 'バッドばつ丸', 'ワンピース',
       'SLAM DUNK', 'ガンダム', 'コボちゃん', 'カピバラさん', 'タイムボカン', 'リロ＆スティッチ',
       'BORUTO', 'PUI PUI モルカー', 'ちびまる子ちゃん', 'コナン', 'ぼのぼの', 'タマ＆フレンズ',
       '遊☆戯☆王

In [90]:
rank_characters(['fan_NARUTO'], False, False) 

1 -> 1


array(['炎炎ノ消防隊', 'ハイキュー!!', '暗殺教室', 'あひるの空', 'Ｄｒ．ＳＴＯＮＥ', 'スパイダーマン', '犬夜叉',
       'BORUTO', '黒子のバスケ', 'メジャー', 'カンフー・パンダ', 'おさるのジョージ', 'ペネロペ', 'ルルロロ',
       '食戟のソーマ', 'ワンピース', 'しまじろう', 'のりものまん モービルランドのカークン', '弱虫ペダル', 'DAYS',
       'EDENS ZERO', 'ミッフィー', 'ティニー', 'FAIRY TAIL', 'チャギントン', 'NARUTO',
       'ミュークルドリーミー', 'ミッキーマウス', 'はなかっぱ', 'おじゃる丸', '妖怪ウォッチ', 'タンタン',
       'iiiあいすくりん', 'トムとジェリー', 'コラショ', '僕のヒーローアカデミア', 'マジカパーティー',
       'かいけつゾロリ', 'キューティーハニー', 'こちら葛飾区亀有公園前派出所', 'アナと雪の女王', 'きかんしゃトーマス',
       'ワールドトリガー', '七つの大罪', 'ペッパピッグ＆パジャマスク-', 'コナン',
       'かいじゅうステップ\u3000ワンダバダ', 'スーパー戦隊', 'ウサハナ', 'LEGO', '忍たま乱太郎',
       'イナズマイレブン', 'レイトン教授', 'おとうさんといっしょ', 'おかあさんといっしょ', 'テニスの王子様',
       'シナモン', 'トイストーリー', '少年アシベ ', 'ピングー', 'モンスターズインク', 'ムーミン', 'WASIMO',
       'SONIC', 'ハローキティ', '美少女戦士セーラームーン', 'ケロロ軍曹', 'デュエル･マスターズ', '鷹の爪',
       'プリキュア', 'リロ＆スティッチ', 'タイムボカン', 'るろうに剣心－明治剣客浪漫譚－', 'けろけろけろっぴ',
       '仮面ライダー', 'プリティーシリーズ', 'マイメロディ', 'くまのプーさん', 'みいつけた！', 'スポンジ・ボブ',
       'アイカツ', 'おジャ魔

In [93]:
rank_characters(['fan_ハローキティ'], False, True) 

1 -> 1


array(['ハローキティ', 'ペネロペ', 'ルルロロ', 'ミッフィー', 'しまじろう', 'ティニー',
       'ペッパピッグ＆パジャマスク-', 'ミュークルドリーミー', 'プリキュア', 'iiiあいすくりん', 'いないいないばあっ！',
       'おかあさんといっしょ', 'マイメロディ', 'おジャ魔女どれみ', 'みいつけた！', 'はなかっぱ', 'アンパンマン',
       'おしりかじり虫', '宇宙なんちゃら こてつくん', 'ピングー', 'ムーミン', 'シナモン', 'ウサハナ', 'ぼのぼの',
       'おしりたんてい', 'おとうさんといっしょ', 'のりものまん モービルランドのカークン', 'ワールドトリガー',
       '炎炎ノ消防隊', 'ミッキーマウス', 'アイカツ', 'BLEACH', 'アナと雪の女王', 'BORUTO',
       'プリティーシリーズ', 'おじゃる丸', '少年アシベ ', 'ちびまる子ちゃん', 'コラショ', 'パウ・パトロール',
       'けろけろけろっぴ', 'こびと', '美少女戦士セーラームーン', 'おさるのジョージ', 'ダイヤのA', 'Ｄｒ．ＳＴＯＮＥ',
       'ハイキュー!!', 'あたしンち', 'かいじゅうステップ\u3000ワンダバダ', 'NARUTO', 'セサミ',
       'WASIMO', '食戟のソーマ', 'くまのプーさん', '聖闘士星矢', 'バッドばつ丸',
       'リトルツインスターズ(キキ＆ララ)', 'ふしぎ駄菓子屋 銭天堂', 'FAIRY TAIL', 'キテレツ', '忍たま乱太郎',
       'こちら葛飾区亀有公園前派出所', 'レイトン教授', 'リロ＆スティッチ', '遊☆戯☆王', '黒子のバスケ',
       'ドラゴンボール', 'チャギントン', 'ブラッククローバー', 'キューティーハニー', 'かいけつゾロリ', 'カピバラさん',
       'キン肉マン', 'タイムボカン', '銀河鉄道999', 'コボちゃん', 'タマ＆フレンズ', 'PUI PUI モルカー',
       'カードキャプターさくら', 'あひ

In [ ]:
rank_characters(['fan_ドラゴンボール']) 

# run on fake + real data kids only - als

X: NARUTO

In [89]:
model = Ranker('als_model_kids_only.2021-12-06')  # 

In [90]:
rank_characters(['fan_アンパンマン'], False)  # 'fan_アンパンマン', 'fan_妖怪ウォッチ', 'fan_NARUTO', 'fan_ハローキティ', fan_ドラゴンボール

1 -> 1


array(['アンパンマン', 'ペッパピッグ＆パジャマスク-', '炎炎ノ消防隊', '幽☆遊☆白書', 'ブラッククローバー',
       'シャーマンキング', 'EDENS ZERO', 'かいじゅうステップ\u3000ワンダバダ', 'キテレツ', 'コボちゃん',
       'バッドばつ丸', 'おじゃる丸', '忍たま乱太郎', 'ミッキーマウス', 'ダイヤのA', 'ピングー',
       'けろけろけろっぴ', 'DRAGON QUEST ダイの大冒険', 'キューティーハニー', 'SLAM DUNK',
       'デジモンアドベンチャー', 'おしりたんてい', 'ハイキュー!!', 'ゲゲゲの鬼太郎', 'ケロロ軍曹', 'ミニオンズ',
       '弱虫ペダル', 'リトルツインスターズ(キキ＆ララ)', 'みいつけた！', 'くまのプーさん', 'はなかっぱ',
       '七つの大罪', 'しまじろう', 'ドゲンジャーズ', 'プリティーシリーズ', 'アイカツ', 'ウサビッチ', '暗殺教室',
       'ぼのぼの', 'おそ松くん', 'NARUTO', 'PUI PUI モルカー', 'WASIMO', 'イナズマイレブン',
       '遊☆戯☆王', '天才バカボン', 'レイトン教授', 'ハローキティ', 'ドラゴンボール', 'おかあさんといっしょ',
       'おさるのジョージ', '仮面ライダー', 'メジャー', 'モンスターズインク', 'トムとジェリー', 'iiiあいすくりん',
       '妖怪ウォッチ', 'コナン', 'スポンジ・ボブ', 'テニスの王子様', 'のりものまん モービルランドのカークン',
       'タマ＆フレンズ', '美少女戦士セーラームーン', 'ウサハナ', '犬夜叉', 'ワンピース', 'ルパン3世',
       'きかんしゃトーマス', 'いないいないばあっ！', 'プリキュア', '少年アシベ ', 'こちら葛飾区亀有公園前派出所',
       'リロ＆スティッチ', 'シナモン', 'こびと', 'ワールドトリガー', 'かいけつゾロリ', 'スーパー戦隊',
       'タイムボカン', 'スパイ

In [92]:
rank_characters(['fan_妖怪ウォッチ']) 

1 -> 1


array(['妖怪ウォッチ', 'マジカパーティー', 'ゲゲゲの鬼太郎', 'ムーミン', 'はなかっぱ',
       'かいじゅうステップ\u3000ワンダバダ', 'カードキャプターさくら', 'ブラッククローバー', 'ペネロペ',
       'あひるの空', 'タンタン', 'DAYS', 'ミッフィー', 'パズドラ', 'ミッキーマウス', 'コラショ',
       'おとうさんといっしょ', 'ワールドトリガー', 'モンスターズインク', 'BLEACH', 'スポンジ・ボブ',
       'ドゲンジャーズ', 'ピングー', 'キューティーハニー', 'しまじろう', 'こびと', 'FAIRY TAIL',
       'バッドばつ丸', 'ケロロ軍曹', 'シナモン', '黒子のバスケ', 'あたしンち', 'LEGO', 'アイカツ',
       'BORUTO', '遊☆戯☆王', 'トイストーリー', '鉄腕アトム', 'ミニオンズ', 'けろけろけろっぴ',
       'HUNTER×HUNTER', 'ウルトラマン', 'くまのプーさん', '七つの大罪', 'チャギントン', 'プリキュア',
       '宇宙なんちゃら こてつくん', 'るろうに剣心－明治剣客浪漫譚－', '美少女戦士セーラームーン', 'いないいないばあっ！',
       'WASIMO', 'セサミ', 'NARUTO', 'ワンピース', 'トムとジェリー', 'ドラゴンボール', 'スーパー戦隊',
       'テニスの王子様', 'リトルツインスターズ(キキ＆ララ)', 'ダイヤのA', 'ルパン3世', '銀河鉄道999',
       'こちら葛飾区亀有公園前派出所', 'おかあさんといっしょ', 'きかんしゃトーマス', '仮面ライダー', 'ルルロロ',
       'アンパンマン', '弱虫ペダル', 'キン肉マン', 'ハローキティ', 'メジャー', 'みいつけた！', 'ウサビッチ',
       'コナン', '忍たま乱太郎', '犬夜叉', 'ハイキュー!!', 'マイメロディ', '僕のヒーローアカデミア',
       'デジモンアドベンチャー', 'イナズマイレブン', 'ウサハ

In [93]:
rank_characters(['fan_NARUTO']) 

1 -> 1


array(['ふしぎ駄菓子屋 銭天堂', 'BLEACH', 'かいじゅうステップ\u3000ワンダバダ', 'iiiあいすくりん',
       '宇宙なんちゃら こてつくん', 'ペッパピッグ＆パジャマスク-', '天才バカボン', 'キテレツ', 'スパイダーマン',
       'あひるの空', 'コボちゃん', 'ルルロロ', 'バッドばつ丸', 'おとうさんといっしょ', 'DAYS',
       'カードキャプターさくら', 'ミッキーマウス', 'ペネロペ', 'レイトン教授', 'モンスターズインク',
       'デュエル･マスターズ', 'ちびまる子ちゃん', 'LEGO', 'くまのプーさん', 'ブラッククローバー',
       'ドゲンジャーズ', 'ケロロ軍曹', 'SONIC', 'マジカパーティー', 'みいつけた！', '忍たま乱太郎',
       'NARUTO', '幽☆遊☆白書', 'デジモンアドベンチャー', 'コラショ', 'セサミ', 'ウサハナ',
       'リトルツインスターズ(キキ＆ララ)', 'FAIRY TAIL', 'マイメロディ', 'はなかっぱ', 'ティニー',
       'シナモン', '鉄腕アトム', 'PUI PUI モルカー', 'けろけろけろっぴ', 'ウサビッチ', 'ミニオンズ',
       'チャギントン', 'タマ＆フレンズ', 'ピングー', 'ベイブレード', 'おジャ魔女どれみ', 'トムとジェリー',
       'ウルトラマン', 'イナズマイレブン', 'タイムボカン', 'おじゃる丸', 'おそ松くん', '食戟のソーマ',
       '弱虫ペダル', 'ワンピース', 'こちら葛飾区亀有公園前派出所', 'ドラゴンボール', 'こびと', '仮面ライダー',
       'プリキュア', '聖闘士星矢', 'しまじろう', 'ルパン3世', 'テニスの王子様', 'SLAM DUNK', 'メジャー',
       'スーパー戦隊', 'WASIMO', 'ワールドトリガー', 'アンパンマン', '妖怪ウォッチ', 'きかんしゃトーマス',
       'パズドラ', '少年アシベ ', '犬夜叉', 'トイストーリー

In [94]:
rank_characters(['fan_ハローキティ']) 

1 -> 1


array(['ウサハナ', 'ハローキティ', 'シナモン', 'ルルロロ', 'ミッフィー', 'モンスターズインク', 'ペネロペ',
       '炎炎ノ消防隊', 'スポンジ・ボブ', '黒子のバスケ', 'ミュークルドリーミー', 'ウサビッチ',
       'リトルツインスターズ(キキ＆ララ)', 'おとうさんといっしょ', 'WASIMO', 'キテレツ', 'タイムボカン',
       'ポケットモンスター', 'マイメロディ', 'ティニー', 'シャーマンキング', 'みいつけた！', 'キン肉マン',
       'しまじろう', '少年アシベ ', 'ブラッククローバー', 'HUNTER×HUNTER', '銀河鉄道999',
       'プリティーシリーズ', 'ゲゲゲの鬼太郎', 'コラショ', 'デジモンアドベンチャー', 'ガンダム',
       '美少女戦士セーラームーン', 'かいけつゾロリ', '食戟のソーマ', '天才バカボン', '鷹の爪', 'Ｄｒ．ＳＴＯＮＥ',
       'チャギントン', 'おしりたんてい', '犬夜叉', 'スーパー戦隊', 'PUI PUI モルカー', '仮面ライダー',
       'のりものまん モービルランドのカークン', '暗殺教室', 'ドゲンジャーズ', 'バッドばつ丸',
       'DRAGON QUEST ダイの大冒険', 'コナン', 'トムとジェリー', 'メジャー', 'コボちゃん', 'ルパン3世',
       'アンパンマン', 'あたしンち', 'セサミ', 'ウルトラマン', 'ワンピース', 'ワールドトリガー', '鉄腕アトム',
       'ピングー', '僕のヒーローアカデミア', 'プリキュア', 'SLAM DUNK', 'FAIRY TAIL', '聖闘士星矢',
       'NARUTO', 'カンフー・パンダ', 'ドラゴンボール', 'カピバラさん', 'テニスの王子様', '妖怪ウォッチ',
       'きかんしゃトーマス', 'おそ松くん', 'イナズマイレブン', 'こちら葛飾区亀有公園前派出所', '遊☆戯☆王',
       'ベイブレード', 'おかあさんといっしょ', 'DAYS

In [95]:
rank_characters(['fan_ドラゴンボール']) 

1 -> 1


array(['ドラゴンボール', 'あひるの空', '宇宙なんちゃら こてつくん', 'キテレツ', 'ペッパピッグ＆パジャマスク-',
       'ペネロペ', 'ムーミン', 'カンフー・パンダ', 'アナと雪の女王', 'みいつけた！',
       'DRAGON QUEST ダイの大冒険', 'ティニー', 'コラショ', 'タンタン', 'モンスターズインク',
       'バッドばつ丸', 'キューティーハニー', 'のりものまん モービルランドのカークン', 'はなかっぱ', 'BLEACH',
       'EDENS ZERO', 'Ｄｒ．ＳＴＯＮＥ', '天才バカボン', 'FAIRY TAIL', 'おそ松くん',
       'ワールドトリガー', 'スパイダーマン', 'ポケットモンスター', 'ミュークルドリーミー', 'ゲゲゲの鬼太郎',
       'かいじゅうステップ\u3000ワンダバダ', 'おしりたんてい', 'パウ・パトロール', 'パズドラ', 'ウサビッチ',
       '忍たま乱太郎', 'キン肉マン', 'おしりかじり虫', 'ハイキュー!!', 'おジャ魔女どれみ', 'コボちゃん',
       'アイカツ', '炎炎ノ消防隊', 'ドゲンジャーズ', 'HUNTER×HUNTER', '妖怪ウォッチ',
       'PUI PUI モルカー', '遊☆戯☆王', '七つの大罪', '聖闘士星矢', 'シナモン', 'おとうさんといっしょ',
       'ダイヤのA', 'トムとジェリー', 'カピバラさん', '暗殺教室', 'トイストーリー', 'プリキュア',
       'きかんしゃトーマス', '弱虫ペダル', 'ウサハナ', 'コナン', 'ウルトラマン', 'けろけろけろっぴ', 'ワンピース',
       'アンパンマン', 'ルパン3世', '美少女戦士セーラームーン', 'NARUTO', 'メジャー', '犬夜叉',
       '仮面ライダー', 'おさるのジョージ', 'イナズマイレブン', 'おかあさんといっしょ', 'セサミ', 'LEGO',
       'いないいないばあっ！', 'ハローキティ', 'こちら葛飾区亀有公園前派出所', 

# run on fake + real data kids only - bpr

In [15]:
model = Ranker('bpr_model_kids_only.2021-12-06')  # 

In [54]:
rank_characters(['fan_ハローキティ'])  # 'fan_アンパンマン', 'fan_妖怪ウォッチ', 'fan_NARUTO', 'fan_ハローキティ', fan_ドラゴンボール

1 -> 1


array(['ペネロペ', 'ルルロロ', 'ミッフィー', 'しまじろう', 'ティニー', 'ペッパピッグ＆パジャマスク-',
       'ミュークルドリーミー', 'プリキュア', 'iiiあいすくりん', 'いないいないばあっ！', 'おかあさんといっしょ',
       'マイメロディ', 'おジャ魔女どれみ', 'みいつけた！', 'はなかっぱ', 'アンパンマン', 'おしりかじり虫',
       '宇宙なんちゃら こてつくん', 'ピングー', 'ムーミン', 'シナモン', 'ウサハナ', 'ぼのぼの', 'おしりたんてい',
       'おとうさんといっしょ', 'のりものまん モービルランドのカークン', 'ワールドトリガー', '炎炎ノ消防隊',
       'ミッキーマウス', 'アイカツ', 'ハローキティ', 'BLEACH', 'アナと雪の女王', 'BORUTO',
       'プリティーシリーズ', 'おじゃる丸', '少年アシベ ', 'ちびまる子ちゃん', 'コラショ', 'パウ・パトロール',
       'けろけろけろっぴ', 'こびと', '美少女戦士セーラームーン', 'おさるのジョージ', 'ダイヤのA', 'Ｄｒ．ＳＴＯＮＥ',
       'ハイキュー!!', 'あたしンち', 'かいじゅうステップ\u3000ワンダバダ', 'NARUTO', 'セサミ',
       'WASIMO', '食戟のソーマ', 'くまのプーさん', '聖闘士星矢', 'バッドばつ丸',
       'リトルツインスターズ(キキ＆ララ)', 'ふしぎ駄菓子屋 銭天堂', 'FAIRY TAIL', 'キテレツ', '忍たま乱太郎',
       'こちら葛飾区亀有公園前派出所', 'レイトン教授', 'リロ＆スティッチ', '遊☆戯☆王', '黒子のバスケ',
       'ドラゴンボール', 'チャギントン', 'ブラッククローバー', 'キューティーハニー', 'かいけつゾロリ', 'カピバラさん',
       'キン肉マン', 'タイムボカン', '銀河鉄道999', 'コボちゃん', 'タマ＆フレンズ', 'PUI PUI モルカー',
       'カードキャプターさくら', 'あひ

# run on fake weight 3 + real data - als

x: NARUTO & ハローキティ

In [76]:
model = Ranker('als_model_w3.2021-12-07')  # 

In [77]:
rank_characters(['fan_アンパンマン'])

1 -> 1


array(['アンパンマン', 'しまじろう', 'パウ・パトロール', 'ケロロ軍曹', 'おさるのジョージ', 'おしりたんてい',
       'プリキュア', 'ペッパピッグ＆パジャマスク-', 'HUNTER×HUNTER', 'いないいないばあっ！',
       'トムとジェリー', 'ワールドトリガー', 'ゲゲゲの鬼太郎', 'ペネロペ', 'おそ松くん', '暗殺教室', 'はなかっぱ',
       '弱虫ペダル', 'コラショ', 'ミッキーマウス', 'かいけつゾロリ', 'アナと雪の女王', '僕のヒーローアカデミア',
       'おジャ魔女どれみ', '美少女戦士セーラームーン', 'ミニオンズ', '忍たま乱太郎', 'きかんしゃトーマス',
       'トイストーリー', 'チャギントン', 'けろけろけろっぴ', 'SONIC', 'ハローキティ', 'ドラゴンボール',
       'のりものまん モービルランドのカークン', 'ミッフィー', 'ルルロロ', 'ポケットモンスター', '食戟のソーマ',
       'ルパン3世', 'DAYS', 'ティニー', 'ウサハナ', 'こちら葛飾区亀有公園前派出所', 'デジモンアドベンチャー',
       '犬夜叉', 'ガンダム', 'リトルツインスターズ(キキ＆ララ)', 'カンフー・パンダ', 'キン肉マン',
       'リロ＆スティッチ', 'モンスターズインク', 'ウルトラマン', 'キューティーハニー', 'くまのプーさん', 'キテレツ',
       'タイムボカン', 'みいつけた！', '銀河鉄道999', '鉄腕アトム', 'かいじゅうステップ\u3000ワンダバダ',
       'コナン', 'イナズマイレブン', '天才バカボン', 'おかあさんといっしょ', 'テニスの王子様', 'タマ＆フレンズ',
       'ワンピース', '妖怪ウォッチ', 'コボちゃん', 'PUI PUI モルカー', 'ベイブレード', 'WASIMO',
       '鷹の爪', 'メジャー', '聖闘士星矢', 'パズドラ', 'カピバラさん', 'シナモン', 'NARUTO',
       'ハイキュー!!', '

In [78]:
rank_characters(['fan_妖怪ウォッチ']) 

1 -> 1


array(['ポケットモンスター', '妖怪ウォッチ', 'ふしぎ駄菓子屋 銭天堂', 'おしりたんてい', 'マジカパーティー',
       'スポンジ・ボブ', 'かいけつゾロリ', 'イナズマイレブン', 'パウ・パトロール', 'トムとジェリー',
       '宇宙なんちゃら こてつくん', 'ペッパピッグ＆パジャマスク-', '暗殺教室', 'デュエル･マスターズ', 'コラショ',
       'こびと', 'SONIC', 'ゲゲゲの鬼太郎', 'ケロロ軍曹', 'ミニオンズ', '七つの大罪', 'LEGO',
       'おさるのジョージ', 'キテレツ', 'のりものまん モービルランドのカークン', 'ドラゴンボール', 'レイトン教授',
       'ベイブレード', 'おしりかじり虫', 'ウサビッチ', 'はなかっぱ', 'パズドラ', 'あたしンち', 'おそ松くん',
       'かいじゅうステップ\u3000ワンダバダ', '忍たま乱太郎', 'Ｄｒ．ＳＴＯＮＥ', 'プリキュア', '炎炎ノ消防隊',
       'ウルトラマン', 'カンフー・パンダ', '仮面ライダー', 'アイカツ', 'ミュークルドリーミー', '少年アシベ ',
       'スーパー戦隊', 'DRAGON QUEST ダイの大冒険', '鷹の爪', 'プリティーシリーズ', '僕のヒーローアカデミア',
       'iiiあいすくりん', '黒子のバスケ', 'WASIMO', 'しまじろう', 'デジモンアドベンチャー', 'ドゲンジャーズ',
       '天才バカボン', 'スパイダーマン', 'DAYS', 'おじゃる丸', 'チャギントン', 'メジャー', '弱虫ペダル',
       'キン肉マン', 'けろけろけろっぴ', '鉄腕アトム', 'NARUTO', 'バッドばつ丸', 'ワンピース',
       'SLAM DUNK', 'ガンダム', 'コボちゃん', 'カピバラさん', 'タイムボカン', 'リロ＆スティッチ',
       'BORUTO', 'PUI PUI モルカー', 'ちびまる子ちゃん', 'コナン', 'ぼのぼの', 'タマ＆フレンズ',
       '遊☆戯☆王

In [79]:
rank_characters(['fan_NARUTO']) 

1 -> 1


array(['ハイキュー!!', 'あひるの空', '炎炎ノ消防隊', 'カンフー・パンダ', '美少女戦士セーラームーン',
       'スパイダーマン', 'DAYS', 'EDENS ZERO', 'ルルロロ', 'Ｄｒ．ＳＴＯＮＥ', '七つの大罪',
       'ルパン3世', 'ミュークルドリーミー', 'タンタン', 'おさるのジョージ', 'ペネロペ', 'ムーミン', 'メジャー',
       '僕のヒーローアカデミア', 'ミッキーマウス', 'ワールドトリガー', 'のりものまん モービルランドのカークン',
       'FAIRY TAIL', 'ティニー', 'キューティーハニー', 'チャギントン', 'ミッフィー', '鷹の爪',
       'iiiあいすくりん', 'ワンピース', 'ペッパピッグ＆パジャマスク-', 'プリキュア', 'おかあさんといっしょ',
       '聖闘士星矢', 'ピングー', 'トイストーリー', 'LEGO', 'アナと雪の女王', 'モンスターズインク',
       '仮面ライダー', 'おとうさんといっしょ', 'スーパー戦隊', 'しまじろう', 'きかんしゃトーマス', 'リロ＆スティッチ',
       'かいじゅうステップ\u3000ワンダバダ', 'アイカツ', 'コラショ', '黒子のバスケ', 'ダイヤのA', 'シナモン',
       'はなかっぱ', 'ふしぎ駄菓子屋 銭天堂', 'プリティーシリーズ', 'おじゃる丸', 'ウサハナ', '忍たま乱太郎',
       'おジャ魔女どれみ', 'みいつけた！', '幽☆遊☆白書', 'ぼのぼの', '暗殺教室', '少年アシベ ',
       'くまのプーさん', '銀河鉄道999', 'ハローキティ', 'マジカパーティー', 'カードキャプターさくら',
       '宇宙なんちゃら こてつくん', 'けろけろけろっぴ', 'こちら葛飾区亀有公園前派出所', 'ドゲンジャーズ', 'WASIMO',
       '鉄腕アトム', 'タイムボカン', 'ガンダム', 'おしりかじり虫', 'SONIC', 'セサミ', 'レイトン教授',
       '天才バカボン', 'ウルトラ

In [80]:
rank_characters(['fan_ハローキティ']) 

1 -> 1


array(['ペネロペ', 'ルルロロ', 'ミッフィー', 'しまじろう', 'ティニー', 'ペッパピッグ＆パジャマスク-',
       'いないいないばあっ！', 'プリキュア', 'ミュークルドリーミー', 'おかあさんといっしょ', 'iiiあいすくりん',
       'マイメロディ', 'みいつけた！', 'おジャ魔女どれみ', 'はなかっぱ', 'ワールドトリガー', 'アンパンマン',
       'BLEACH', 'おしりかじり虫', '宇宙なんちゃら こてつくん', 'ピングー', 'シナモン', 'ウサハナ',
       'おしりたんてい', 'ムーミン', 'おとうさんといっしょ', 'のりものまん モービルランドのカークン', 'ミッキーマウス',
       'ハローキティ', '炎炎ノ消防隊', 'ぼのぼの', 'アイカツ', 'Ｄｒ．ＳＴＯＮＥ', 'おじゃる丸', 'コラショ',
       'アナと雪の女王', 'プリティーシリーズ', '少年アシベ ', 'FAIRY TAIL', 'おさるのジョージ',
       'ハイキュー!!', 'けろけろけろっぴ', 'パウ・パトロール', 'ちびまる子ちゃん', '美少女戦士セーラームーン',
       'セサミ', 'かいじゅうステップ\u3000ワンダバダ', 'WASIMO', '弱虫ペダル', '黒子のバスケ', 'こびと',
       '忍たま乱太郎', 'くまのプーさん', 'バッドばつ丸', 'ブラッククローバー', '聖闘士星矢',
       'リトルツインスターズ(キキ＆ララ)', 'ふしぎ駄菓子屋 銭天堂', 'あひるの空', 'イナズマイレブン', 'レイトン教授',
       'テニスの王子様', '遊☆戯☆王', 'ダイヤのA', 'あたしンち', 'るろうに剣心－明治剣客浪漫譚－',
       'リロ＆スティッチ', 'ドラゴンボール', 'かいけつゾロリ', '食戟のソーマ', 'キューティーハニー', 'DAYS',
       'ルパン3世', 'チャギントン', '銀河鉄道999', 'カピバラさん', 'タイムボカン', 'タマ＆フレンズ',
       'EDENS ZERO', 'PUI PU

In [81]:
rank_characters(['fan_ドラゴンボール']) 

1 -> 1


array(['ドラゴンボール', '幽☆遊☆白書', 'キテレツ', 'DRAGON QUEST ダイの大冒険', 'SLAM DUNK',
       'HUNTER×HUNTER', 'るろうに剣心－明治剣客浪漫譚－', 'こちら葛飾区亀有公園前派出所', '聖闘士星矢',
       'ゲゲゲの鬼太郎', 'イナズマイレブン', 'キン肉マン', 'トムとジェリー', 'ケロロ軍曹', 'ちびまる子ちゃん',
       '僕のヒーローアカデミア', '美少女戦士セーラームーン', 'SONIC', 'かいけつゾロリ', '妖怪ウォッチ',
       '忍たま乱太郎', 'おジャ魔女どれみ', '天才バカボン', 'NARUTO', 'ぼのぼの', 'おそ松くん',
       'アンパンマン', 'カンフー・パンダ', 'ベイブレード', 'おじゃる丸', 'ハイキュー!!', '黒子のバスケ',
       'デジモンアドベンチャー', 'ムーミン', 'ルパン3世', 'スポンジ・ボブ', 'はなかっぱ', 'ウルトラマン',
       '鷹の爪', 'あたしンち', 'キューティーハニー', '暗殺教室', 'リロ＆スティッチ', 'タンタン', '少年アシベ ',
       '遊☆戯☆王', 'トイストーリー', 'コボちゃん', 'デュエル･マスターズ', '仮面ライダー', 'コラショ',
       'ミッキーマウス', 'バッドばつ丸', 'マイメロディ', 'けろけろけろっぴ', 'くまのプーさん', 'WASIMO',
       'パズドラ', 'PUI PUI モルカー', 'タイムボカン', '鉄腕アトム', 'マジカパーティー',
       'おかあさんといっしょ', 'モンスターズインク', 'こびと', 'リトルツインスターズ(キキ＆ララ)', 'ハローキティ',
       'おしりたんてい', 'おしりかじり虫', 'みいつけた！', 'スーパー戦隊', 'タマ＆フレンズ', 'LEGO',
       'シナモン', 'カピバラさん', 'おとうさんといっしょ', 'セサミ', 'ワンピース', 'プリキュア', 'ウサハナ',
       'ガンダム', 'ミニオンズ', 'きかん

In [86]:
rank_characters(['PM015288972'], True) 

1 -> 1


'SID0018518,SID0041925,SID0042040,SID0034919,SID0002629,SID0010889,SID0059025,SID0002812,SID0023813,SID0050942,SID0053718,SID0028806,SID0002160,SID0010865,SID0002794,SID0002377,SID0002374,SID0047137,SID0014278,SID0012244,SID0031542,SID0018228,SID0003708,SID0012963,SID0044433,SID0002202,SID0015965,SID0020415,SID0031251,SID0014719,SID0022028,SID0012095,SID0018369,SID0003430,SID0002301,SID0002236,SID0012119,SID0011051,SID0002500,SID0011154,SID0015493,SID0047529,SID0020005,SID0056089,SID0002499,SID0044348,SID0011633,SID0025580,SID0051059,SID0015840,SID0020003,SID0002372,SID0003094,SID0035447,SID0056092,SID0011447,SID0056132,SID0022136,SID0043840,SID0013588,SID0020016,SID0013572,SID0002766,SID0002339,SID0020008,SID0011460,SID0003252,SID0011682,SID0047155,SID0003728,SID0014642,SID0014066,SID0011124,SID0017187,SID0004917,SID0002142,SID0017681,SID0015605,SID0012681,SID0002347,SID0002319,SID0012621,SID0051161,SID0023753,SID0025664,SID0010922,SID0058493,SID0002237,SID0003875,SID0002345,SID000235

# run on fake weight 3 + real data - bpr

In [63]:
model = Ranker('bpr_model_w3.2021-12-07')  # 

In [56]:
rank_characters(['fan_アンパンマン'])  # 'fan_アンパンマン', 'fan_妖怪ウォッチ', 'fan_NARUTO', 'fan_ハローキティ', fan_ドラゴンボール

1 -> 1


array(['パウ・パトロール', 'ペネロペ', 'アンパンマン', 'チャギントン', 'おしりたんてい', 'iiiあいすくりん',
       'しまじろう', 'ミッフィー', 'ミッキーマウス', 'きかんしゃトーマス', 'ルルロロ',
       'のりものまん モービルランドのカークン', 'はなかっぱ', 'いないいないばあっ！', 'シナモン', 'ピングー',
       'ティニー', 'けろけろけろっぴ', 'おさるのジョージ', 'リロ＆スティッチ', 'ハローキティ', 'ミュークルドリーミー',
       'コラショ', 'みいつけた！', 'くまのプーさん', 'リトルツインスターズ(キキ＆ララ)', 'カピバラさん',
       'ペッパピッグ＆パジャマスク-', 'おかあさんといっしょ', 'かいじゅうステップ\u3000ワンダバダ', 'セサミ',
       'おしりかじり虫', 'ウサハナ', 'こびと', 'アナと雪の女王', 'WASIMO', 'ウサビッチ',
       'おとうさんといっしょ', '宇宙なんちゃら こてつくん', 'トイストーリー', 'プリキュア', '少年アシベ ',
       'マイメロディ', 'かいけつゾロリ', 'トムとジェリー', 'ミニオンズ', 'タマ＆フレンズ', 'ぼのぼの',
       'マジカパーティー', 'PUI PUI モルカー', 'ポケットモンスター', 'スポンジ・ボブ', 'ムーミン',
       'SONIC', 'おジャ魔女どれみ', 'ゲゲゲの鬼太郎', 'おじゃる丸', 'ドゲンジャーズ', 'モンスターズインク',
       'LEGO', 'バッドばつ丸', 'ふしぎ駄菓子屋 銭天堂', '忍たま乱太郎', 'キテレツ', '妖怪ウォッチ',
       'ちびまる子ちゃん', 'プリティーシリーズ', 'ウルトラマン', '美少女戦士セーラームーン', 'コボちゃん', 'アイカツ',
       'あたしンち', '鉄腕アトム', 'パズドラ', 'ケロロ軍曹', 'スーパー戦隊', '天才バカボン', 'タイムボカン',
       'カンフー・パンダ', 'キン肉マン', '仮面ライダー', '

In [57]:
rank_characters(['fan_妖怪ウォッチ']) 

1 -> 1


array(['妖怪ウォッチ', 'ポケットモンスター', 'マジカパーティー', 'SONIC', 'かいけつゾロリ', 'パウ・パトロール',
       'おしりたんてい', 'スポンジ・ボブ', 'ウサビッチ', 'こびと', 'トムとジェリー', 'コラショ', 'カピバラさん',
       'ミニオンズ', 'リロ＆スティッチ', 'iiiあいすくりん', 'おさるのジョージ', 'ミュークルドリーミー',
       '少年アシベ ', 'パズドラ', '宇宙なんちゃら こてつくん', 'おしりかじり虫', 'ゲゲゲの鬼太郎', 'WASIMO',
       'LEGO', 'はなかっぱ', 'ふしぎ駄菓子屋 銭天堂', 'ペッパピッグ＆パジャマスク-', 'ペネロペ',
       'のりものまん モービルランドのカークン', 'かいじゅうステップ\u3000ワンダバダ', 'デュエル･マスターズ',
       'マイメロディ', 'けろけろけろっぴ', 'チャギントン', 'ピングー', 'バッドばつ丸', 'プリキュア',
       'くまのプーさん', 'ドゲンジャーズ', '忍たま乱太郎', 'ルルロロ', 'PUI PUI モルカー', 'アナと雪の女王',
       'ケロロ軍曹', 'トイストーリー', 'リトルツインスターズ(キキ＆ララ)', 'あたしンち', 'ウルトラマン',
       'ミッキーマウス', 'キテレツ', 'しまじろう', 'ぼのぼの', 'おジャ魔女どれみ', 'ハローキティ', 'おじゃる丸',
       'アイカツ', 'タマ＆フレンズ', 'プリティーシリーズ', 'シナモン', 'カンフー・パンダ', 'ティニー',
       'ミッフィー', 'ウサハナ', 'モンスターズインク', 'ちびまる子ちゃん', 'きかんしゃトーマス', 'ムーミン',
       'セサミ', 'スーパー戦隊', '美少女戦士セーラームーン', 'ベイブレード', 'いないいないばあっ！', '仮面ライダー',
       'みいつけた！', 'おかあさんといっしょ', 'アンパンマン', '天才バカボン', 'コボちゃん', 'イナズマイレブン',
       'おとうさんと

In [58]:
rank_characters(['fan_NARUTO']) 

1 -> 1


array(['ふしぎ駄菓子屋 銭天堂', 'キテレツ', 'ちびまる子ちゃん', 'ぼのぼの', 'ポケットモンスター', 'おじゃる丸',
       '鉄腕アトム', 'コボちゃん', 'キューティーハニー', '銀河鉄道999', 'アナと雪の女王', 'タマ＆フレンズ',
       'あたしンち', '天才バカボン', '少年アシベ ', 'タンタン', 'ムーミン', 'PUI PUI モルカー',
       'レイトン教授', 'ドゲンジャーズ', 'パウ・パトロール', 'ミュークルドリーミー', 'カピバラさん', 'セサミ',
       'ウサビッチ', 'マイメロディ', 'SONIC', 'ブラッククローバー', 'HUNTER×HUNTER',
       '美少女戦士セーラームーン', 'ゲゲゲの鬼太郎', 'おジャ魔女どれみ', 'カードキャプターさくら', 'ミニオンズ',
       'ペネロペ', '忍たま乱太郎', 'キン肉マン', 'BLEACH', '幽☆遊☆白書', 'タイムボカン', 'スポンジ・ボブ',
       '宇宙なんちゃら こてつくん', 'おしりたんてい', 'るろうに剣心－明治剣客浪漫譚－', 'ウルトラマン', 'NARUTO',
       'SLAM DUNK', '黒子のバスケ', 'ケロロ軍曹', 'くまのプーさん', 'DRAGON QUEST ダイの大冒険',
       'バッドばつ丸', 'アイカツ', 'カンフー・パンダ', 'おさるのジョージ', 'トイストーリー',
       'リトルツインスターズ(キキ＆ララ)', 'スパイダーマン', 'トムとジェリー', 'マジカパーティー', 'スーパー戦隊',
       'iiiあいすくりん', 'ミッフィー', 'プリティーシリーズ', 'ミッキーマウス', '仮面ライダー', 'テニスの王子様',
       'Ｄｒ．ＳＴＯＮＥ', 'ピングー', 'おしりかじり虫', 'シナモン', 'かいけつゾロリ', 'BORUTO',
       'モンスターズインク', 'こびと', 'きかんしゃトーマス', 'シャーマンキング', 'コラショ', 'プリキュア',
       'チャギントン', '鷹

In [59]:
rank_characters(['fan_ハローキティ']) 

1 -> 1


array(['パウ・パトロール', 'ペネロペ', 'iiiあいすくりん', 'おしりたんてい', 'チャギントン',
       'のりものまん モービルランドのカークン', 'シナモン', 'ルルロロ', 'はなかっぱ', 'ミュークルドリーミー',
       'ミッフィー', 'ピングー', 'コラショ', 'しまじろう', 'ミッキーマウス', 'きかんしゃトーマス', 'ティニー',
       'けろけろけろっぴ', 'こびと', 'いないいないばあっ！', 'おさるのジョージ', 'ハローキティ', 'カピバラさん',
       'かいじゅうステップ\u3000ワンダバダ', 'リトルツインスターズ(キキ＆ララ)', 'リロ＆スティッチ',
       'ペッパピッグ＆パジャマスク-', 'みいつけた！', 'おしりかじり虫', 'くまのプーさん', 'おかあさんといっしょ',
       'ウサハナ', 'セサミ', 'WASIMO', 'アンパンマン', '宇宙なんちゃら こてつくん', 'ウサビッチ',
       '少年アシベ ', 'アナと雪の女王', 'おとうさんといっしょ', 'かいけつゾロリ', 'マイメロディ', 'プリキュア',
       'トムとジェリー', 'PUI PUI モルカー', 'マジカパーティー', 'トイストーリー', 'ムーミン', 'ぼのぼの',
       'タマ＆フレンズ', 'スポンジ・ボブ', 'ミニオンズ', 'SONIC', 'ドゲンジャーズ', 'ポケットモンスター',
       'ゲゲゲの鬼太郎', 'LEGO', 'おじゃる丸', 'ふしぎ駄菓子屋 銭天堂', 'おジャ魔女どれみ', 'バッドばつ丸',
       '妖怪ウォッチ', 'プリティーシリーズ', '忍たま乱太郎', 'ウルトラマン', 'キテレツ', 'モンスターズインク',
       'ちびまる子ちゃん', 'コボちゃん', 'アイカツ', '美少女戦士セーラームーン', 'あたしンち', '鉄腕アトム',
       'パズドラ', 'スーパー戦隊', '天才バカボン', 'ケロロ軍曹', 'タイムボカン', '仮面ライダー',
       'デュエル･マスターズ', 'カンフー・パン

In [65]:
rank_characters(['fan_ドラゴンボール'], output_sid=True) 

1 -> 1
